In [13]:

import tkinter as tk
from tkinter import ttk, messagebox, filedialog  # Added filedialog here
import pandas as pd
import numpy as np
from datetime import datetime
import os

class BenchmarkPortfolio:
    def __init__(self, securities_data: pd.DataFrame, weights: dict, as_of_date: str = None):
        """
        Calculate benchmark portfolio characteristics
        
        Args:
            securities_data (pd.DataFrame): Securities data
            weights (dict): Dictionary of security weights
            as_of_date (str): Reference date for calculations (YYYY/MM/DD)
        """
        self.securities_data = securities_data.copy()
        self.weights = weights
        
        self.calculate_characteristics()
    
    def calculate_characteristics(self):
        """Calculate weighted characteristics of the benchmark portfolio"""
        # Add weights to dataframe
        self.securities_data['Weight'] = self.securities_data['SecurityId'].astype(str).map(self.weights).fillna(0)
        
        # Calculate weighted characteristics
        self.characteristics = {
            'number_of_securities': len([w for w in self.weights.values() if w > 0]),
            'modified_duration': self._weighted_average('ModifiedDuration'),
            'spread': self._weighted_average('spread'),
            'coupon': self._weighted_average('Coupon'),
            'liquidity': self._weighted_average('LiquidityScore'),
            'clean_price': self._weighted_average('CleanPrice'),
            'interest': self._weighted_average('AccruedInterest'),
            'dirty_price': self._weighted_average('DirtyPrice'),
            'bidask': self._weighted_average('BidAskSpread'),
            'std': self._weighted_average('StdDev'),
            'expected_return': self._weighted_average('ExpectedReturn'),
            'rating_weights': self._calculate_rating_weights(),
            'total_weight': self.securities_data['Weight'].sum()
        }
    
    def _weighted_average(self, column: str) -> float:
        """Calculate weighted average for a given column"""
        return (self.securities_data[column] * self.securities_data['Weight']).sum() / self.securities_data['Weight'].sum()
    
    def _calculate_rating_weights(self) -> dict:
        """Calculate weights by rating"""
        return self.securities_data.groupby('Rating')['Weight'].sum().to_dict()
    
    def get_characteristics(self) -> dict:
        """Return benchmark characteristics"""
        return self.characteristics

class BenchmarkWeightUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Benchmark Weight Input")
        self.root.geometry("800x600")
        
        # Configure weight propagation
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=1)
        
        # Create main frame
        self.main_frame = ttk.Frame(root, padding="10")
        self.main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        self.main_frame.grid_columnconfigure(0, weight=1)
        
        # Data storage
        self.weights_dict = {}
        self.securities_data = None
        
        # Create file upload section
        self.create_file_upload_section()
        
    def create_file_upload_section(self):
        """Create file upload section"""
        upload_frame = ttk.LabelFrame(self.main_frame, text="Data Upload", padding="5")
        upload_frame.grid(row=0, column=0, columnspan=2, sticky=(tk.W, tk.E), pady=5)
        upload_frame.grid_columnconfigure(0, weight=1)
        
        # File path display
        self.file_path_var = tk.StringVar()
        file_path_entry = ttk.Entry(upload_frame, textvariable=self.file_path_var, width=50, state='readonly')
        file_path_entry.grid(row=0, column=0, padx=5, pady=5, sticky=(tk.W, tk.E))
        
        # Upload button
        upload_button = ttk.Button(upload_frame, text="Upload File", command=self.upload_file)
        upload_button.grid(row=0, column=1, padx=5, pady=5)
        
        # Status label
        self.status_var = tk.StringVar()
        status_label = ttk.Label(upload_frame, textvariable=self.status_var)
        status_label.grid(row=1, column=0, columnspan=2, pady=5)
    
    def upload_file(self):
        """Handle file upload"""
        file_types = [
            ('Excel files', '*.xlsx'),
            ('CSV files', '*.csv'),
            ('All files', '*.*')
        ]
        
        try:
            file_path = filedialog.askopenfilename(
                title='Select data file',
                filetypes=file_types,
                initialdir=os.getcwd()  # Start in current directory
            )
            
            if file_path:
                self.file_path_var.set(file_path)
                self.load_data(file_path)
                self.status_var.set("File loaded successfully!")
                
                # Clear existing widgets if they exist
                if hasattr(self, 'weights_frame'):
                    self.weights_frame.destroy()
                if hasattr(self, 'button_frame'):
                    self.button_frame.destroy()
                
                # Create the main UI elements
                self.create_weights_section()
                self.create_buttons()
                
        except Exception as e:
            self.status_var.set(f"Error: {str(e)}")
            messagebox.showerror("Error", f"Error loading file: {str(e)}")
    
    def load_data(self, file_path):
        """Load securities data from file path"""
        try:
            if file_path.lower().endswith('.xlsx'):
                self.securities_data = pd.read_excel(file_path)
            elif file_path.lower().endswith('.csv'):
                self.securities_data = pd.read_csv(file_path)
            else:
                raise ValueError("Unsupported file format. Please use .xlsx or .csv files.")
            
            # Validate required columns
            required_columns = {'SecurityId', 'Rating', 'ModifiedDuration', 'spread', 'Coupon', 
                              'LiquidityScore', 'CleanPrice', 'AccruedInterest', 'DirtyPrice', 
                              'BidAskSpread', 'StdDev', 'ExpectedReturn'}
            
            missing_columns = required_columns - set(self.securities_data.columns)
            if missing_columns:
                raise ValueError(f"Missing required columns: {', '.join(missing_columns)}")
            
            # Convert SecurityId to string
            self.securities_data['SecurityId'] = self.securities_data['SecurityId'].astype(str)
            
            print("Available columns:", self.securities_data.columns.tolist())
            print("\nFirst few rows:")
            print(self.securities_data.head())
            
        except Exception as e:
            self.securities_data = None
            raise e
    
    def create_weights_section(self):
        """Create weights input section"""
        self.weights_frame = ttk.LabelFrame(self.main_frame, text="Benchmark Weights", padding="5")
        self.weights_frame.grid(row=1, column=0, columnspan=2, sticky=(tk.W, tk.E, tk.N, tk.S), pady=5)
        
        # Create scrollable frame for weights
        self.canvas = tk.Canvas(self.weights_frame)
        scrollbar = ttk.Scrollbar(self.weights_frame, orient="vertical", command=self.canvas.yview)
        self.scrollable_frame = ttk.Frame(self.canvas)
        
        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all"))
        )
        
        self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        self.canvas.configure(yscrollcommand=scrollbar.set)
        
        # Pack scrollable components
        self.canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # Add search and filter
        self.filter_frame = ttk.Frame(self.weights_frame)
        self.filter_frame.pack(fill="x", padx=5, pady=5)
        
        # Add Select All checkbox
        self.select_all_var = tk.BooleanVar()
        ttk.Checkbutton(self.filter_frame, text="Select All", 
                       variable=self.select_all_var, 
                       command=self.toggle_all).pack(side="left", padx=5)
        
        ttk.Label(self.filter_frame, text="Search:").pack(side="left", padx=5)
        self.search_var = tk.StringVar()
        self.search_var.trace('w', self.filter_securities)
        ttk.Entry(self.filter_frame, textvariable=self.search_var).pack(side="left", padx=5)
        
        # Create weight entries
        self.create_weight_entries()
    
    def toggle_all(self):
        """Toggle all visible securities"""
        select_all_state = self.select_all_var.get()
        search_text = self.search_var.get().lower()
        
        for i, row in enumerate(self.securities_data.itertuples(), 1):
            sec_id = str(row.SecurityId)
            # Only toggle checkboxes for visible items (matching search filter)
            if search_text in str(row.SecurityId).lower() or search_text in str(row.Rating).lower() or not search_text:
                self.weights_dict[sec_id]['check_var'].set(select_all_state)
    
    def create_weight_entries(self):
        """Create weight entry fields for each security"""
        # Create headers
        headers = ['Select', 'Security ID', 'Rating', 'Weight']
        for col, header in enumerate(headers):
            ttk.Label(self.scrollable_frame, text=header).grid(row=0, column=col, padx=5, pady=5)
        
        # Create entries for each security
        for i, row in enumerate(self.securities_data.itertuples(), 1):
            # Checkbox
            var = tk.BooleanVar()
            ttk.Checkbutton(self.scrollable_frame, variable=var).grid(row=i, column=0)
            
            # Security ID
            ttk.Label(self.scrollable_frame, text=row.SecurityId).grid(row=i, column=1, padx=5)
            
            # Rating
            ttk.Label(self.scrollable_frame, text=row.Rating).grid(row=i, column=2, padx=5)
            
            # Weight entry
            weight_var = tk.StringVar(value="0.0")
            ttk.Entry(self.scrollable_frame, textvariable=weight_var, width=10).grid(row=i, column=3, padx=5)
            
            # Store references
            self.weights_dict[str(row.SecurityId)] = {
                'check_var': var,
                'weight_var': weight_var
            }
    
    def create_buttons(self):
        """Create action buttons"""
        self.button_frame = ttk.Frame(self.main_frame)
        self.button_frame.grid(row=2, column=0, columnspan=2, pady=10)
        
        ttk.Button(self.button_frame, text="Equal Weight Selected", 
                  command=self.equal_weight_selected).pack(side=tk.LEFT, padx=5)
        ttk.Button(self.button_frame, text="Clear All", 
                  command=self.clear_weights).pack(side=tk.LEFT, padx=5)
        ttk.Button(self.button_frame, text="Create Benchmark", 
                  command=self.create_benchmark).pack(side=tk.LEFT, padx=5)
    
    def filter_securities(self, *args):
        """Filter securities based on search text"""
        search_text = self.search_var.get().lower()
        for i, row in enumerate(self.securities_data.itertuples(), 1):
            if search_text in str(row.SecurityId).lower() or search_text in str(row.Rating).lower():
                self.scrollable_frame.grid_slaves(row=i)[0].grid()
            else:
                self.scrollable_frame.grid_slaves(row=i)[0].grid_remove()
        
        # Update select all checkbox based on visible items
        if search_text:
            visible_selected = all(
                self.weights_dict[str(row.SecurityId)]['check_var'].get()
                for row in self.securities_data.itertuples()
                if search_text in str(row.SecurityId).lower() or search_text in str(row.Rating).lower()
            )
            self.select_all_var.set(visible_selected)
    
    def equal_weight_selected(self):
        """Assign equal weights to selected securities"""
        selected = [sec_id for sec_id, vars in self.weights_dict.items() 
                   if vars['check_var'].get()]
        
        if not selected:
            messagebox.showwarning("Warning", "No securities selected")
            return
            
        weight = 1.0 / len(selected)
        for sec_id in selected:
            self.weights_dict[sec_id]['weight_var'].set(f"{weight:.4f}")
    
    def clear_weights(self):
        """Clear all weights and selections"""
        self.select_all_var.set(False)
        for vars in self.weights_dict.values():
            vars['check_var'].set(False)
            vars['weight_var'].set("0.0")
    
    def create_benchmark(self):
        """Create benchmark from input weights"""
        if not self.securities_data is not None:
            messagebox.showerror("Error", "Please upload data file first")
            return
            
        try:
            # Collect weights
            weights = {}
            for sec_id, vars in self.weights_dict.items():
                weight = float(vars['weight_var'].get())
                if weight > 0:
                    weights[sec_id] = weight
            
            # Validate total weight
            total_weight = sum(weights.values())
            if not 0.99 <= total_weight <= 1.01:
                raise ValueError(f"Weights must sum to 1.0 (current sum: {total_weight:.4f})")
            
            # Create benchmark portfolio and calculate characteristics
            benchmark = BenchmarkPortfolio(
                securities_data=self.securities_data,
                weights=weights,
                as_of_date=datetime.now().strftime('%Y/%m/%d')
            )
            self.show_benchmark_results(benchmark)
            
        except Exception as e:
            messagebox.showerror("Error", f"Error creating benchmark: {str(e)}")
    
    def show_benchmark_results(self, benchmark):
            """Show benchmark characteristics in a new window"""
            results_window = tk.Toplevel(self.root)
            results_window.title("Benchmark Portfolio Results")
            results_window.geometry("600x800")
            
            # Make window transient to main window (will always stay on top of main window)
            results_window.transient(self.root)
            
            # Create notebook for tabbed view
            notebook = ttk.Notebook(results_window)
            notebook.pack(fill='both', expand=True, padx=10, pady=10)
            
            # Characteristics tab
            chars_frame = ttk.Frame(notebook)
            notebook.add(chars_frame, text='Portfolio Characteristics')
            
            # Get characteristics
            chars = benchmark.get_characteristics()
            
            # Create text widget for characteristics with scrollbar
            chars_frame_inner = ttk.Frame(chars_frame)
            chars_frame_inner.pack(fill='both', expand=True)
            
            chars_scroll = ttk.Scrollbar(chars_frame_inner)
            chars_scroll.pack(side='right', fill='y')
            
            chars_text = tk.Text(chars_frame_inner, height=30, width=70, yscrollcommand=chars_scroll.set)
            chars_text.pack(side='left', fill='both', expand=True, padx=10, pady=10)
            chars_scroll.config(command=chars_text.yview)
            
            # Insert characteristics with section formatting
            chars_text.tag_configure('header', font=('TkDefaultFont', 10, 'bold'))
            chars_text.tag_configure('section', font=('TkDefaultFont', 9, 'bold'))
            chars_text.tag_configure('normal', font=('TkDefaultFont', 9))
            
            chars_text.insert(tk.END, "Portfolio Characteristics\n\n", 'header')
            chars_text.insert(tk.END, f"Number of Securities: {chars['number_of_securities']}\n", 'normal')
            chars_text.insert(tk.END, f"Total Portfolio Weight: {chars['total_weight']:.2%}\n\n", 'normal')
            
            chars_text.insert(tk.END, "Key Metrics:\n", 'section')
            metrics = [
                ("Modified Duration", chars['modified_duration'], '.2f'),
                ("Spread", chars['spread'], '.2f'),
                ("Coupon", chars['coupon'], '.2f'),
                ("Bid-Ask Spread", chars['bidask'], '.4f'),
                ("Liquidity Score", chars['liquidity'], '.4f'),
                ("Clean Price", chars['clean_price'], '.2f'),
                ("Accrued Interest", chars['interest'], '.4f'),
                ("Dirty Price", chars['dirty_price'], '.2f'),
                ("Standard Deviation", chars['std'], '.4f'),
                ("Expected Return", chars['expected_return'], '.2f')
            ]
            
            for label, value, format_spec in metrics:
                formatted_value = format(value, format_spec)
                chars_text.insert(tk.END, f"{label}: {formatted_value}\n", 'normal')
            
            chars_text.insert(tk.END, "\nRating Distribution:\n", 'section')
            # Sort ratings by weight in descending order
            sorted_ratings = sorted(chars['rating_weights'].items(), key=lambda x: x[1], reverse=True)
            for rating, weight in sorted_ratings:
                chars_text.insert(tk.END, f"{rating}: {weight:.2%}\n", 'normal')
            
            # Weights tab
            weights_frame = ttk.Frame(notebook)
            notebook.add(weights_frame, text='Security Weights')
            
            # Create text widget for weights with scrollbar
            weights_frame_inner = ttk.Frame(weights_frame)
            weights_frame_inner.pack(fill='both', expand=True)
            
            weights_scroll = ttk.Scrollbar(weights_frame_inner)
            weights_scroll.pack(side='right', fill='y')
            
            weights_text = tk.Text(weights_frame_inner, height=30, width=70, yscrollcommand=weights_scroll.set)
            weights_text.pack(side='left', fill='both', expand=True, padx=10, pady=10)
            weights_scroll.config(command=weights_text.yview)
            
            # Insert weights with formatting
            weights_text.tag_configure('header', font=('TkDefaultFont', 10, 'bold'))
            weights_text.tag_configure('normal', font=('TkDefaultFont', 9))
            
            weights_text.insert(tk.END, "Security Weights\n\n", 'header')
            
            # Sort weights by value in descending order
            sorted_weights = sorted(benchmark.weights.items(), key=lambda x: x[1], reverse=True)
            for sec_id, weight in sorted_weights:
                security_info = self.securities_data[self.securities_data['SecurityId'] == sec_id].iloc[0]
                weights_text.insert(tk.END, 
                                f"Security ID: {sec_id}\n"
                                f"Rating: {security_info['Rating']}\n"
                                f"Weight: {weight:.4f}\n"
                                f"Modified Duration: {security_info['ModifiedDuration']:.2f}\n"
                                f"Spread: {security_info['spread']:.2f}\n"
                                f"Expected Return: {security_info['ExpectedReturn']:.2f}\n\n", 
                                'normal')
            
            # Make text widgets read-only
            chars_text.configure(state='disabled')
            weights_text.configure(state='disabled')
            
            # Add export button
            export_frame = ttk.Frame(results_window)
            export_frame.pack(fill='x', padx=10, pady=5)
            
            ttk.Button(export_frame, text="Export Results", 
                    command=lambda: self.export_results(benchmark)).pack(side='right')
            
            # Update the window's minimum size
            results_window.update()
            results_window.minsize(results_window.winfo_width(), results_window.winfo_height())

# Example usage
if __name__ == "__main__":
    root = tk.Tk()
    app = BenchmarkWeightUI(root)
    root.mainloop()

Available columns: ['SecurityId', 'spread', 'Maturity', 'Coupon', 'LiquidityScore', 'CleanPrice', 'AccruedInterest', 'DirtyPrice', 'ModifiedDuration', 'BidAskSpread', 'Rating', 'StdDev', 'ExpectedReturn']

First few rows:
  SecurityId     spread   Maturity  Coupon  LiquidityScore  CleanPrice  \
0  Security1  30.000000 2026-09-30    2.75        0.784261  100.763519   
1  Security2  29.380054 2026-09-30    2.75        0.880383  100.755079   
2  Security3  30.107606 2026-09-30    2.75        1.084863  100.739440   
3  Security4  30.672702 2026-09-30    2.75        0.907092  100.728908   

   AccruedInterest  DirtyPrice  ModifiedDuration  BidAskSpread Rating  \
0         0.700685  101.464204          1.705084      0.062919    AAA   
1         0.821233  101.576312          1.661251      0.070630    AAA   
2         0.843836  101.583276          1.653028      0.087035    AAA   
3         0.851370  101.580278          1.650286      0.072773    AAA   

     StdDev  ExpectedReturn  
0  0.422171

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Melanie\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Melanie\AppData\Local\Temp\ipykernel_28504\2612505442.py", line 430, in <lambda>
    command=lambda: self.export_results(benchmark)).pack(side='right')
                    ^^^^^^^^^^^^^^^^^^^
AttributeError: 'BenchmarkWeightUI' object has no attribute 'export_results'
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Melanie\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Melanie\AppData\Local\Temp\ipykernel_28504\2612505442.py", line 430, in <lambda>
    command=lambda: self.export_results(benchmark)).pack(side='right')
                    ^^^^^^^^^^^^^^^^^^^
AttributeError: 'BenchmarkWeightUI'